<a href="https://colab.research.google.com/github/rafaelsguerra/information_retrieval/blob/master/Lab%2005%20-%20Query%20Expansion/Expans%C3%A3o_de_Consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import collections
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import nltk
import heapq
import time
import math

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/rafaelsguerra/information_retrieval/master/data/results.csv')
data_title = data['title'].tolist()
data_subtitle = data['subtitle'].tolist()
data_text = data['text'].tolist()

In [0]:
def parse(document):  
  tokenizer = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+') 
  tokens = []

  tokens.extend(tokenizer.tokenize(document.lower()))
    
  stop_words = stopwords.words('portuguese')

  filtered_tokens = []

  for token in tokens:
    if token not in stop_words  and len(token) >= 2: # and len(token) > 2
      filtered_tokens.append(token)
  
  return filtered_tokens

In [0]:
def build_index(data):
  inverted_list = {}
  n = 0
  
  for document in data:
    n = n + 1
    tokens = parse(document)
    
    tokens_no_duplicates = collections.Counter(tokens)
    
    for token in tokens_no_duplicates.keys():
      if token not in inverted_list:
        inverted_list[token] = []
      inverted_list[token].append((n, tokens_no_duplicates[token]))
    
  return inverted_list


In [0]:
def document_at_a_time(query, inverted_index, k):
  inverted_lists = {}
  r = []
  
  for word in query.split(" "):
    if word in inverted_index.keys():
      inverted_lists[word] = inverted_index[word]
      
  documents = list(range(1, len(data.url) + 1))
  
  for document in documents:
    score = 0
    
    for key in inverted_lists.keys():
      for tuple in inverted_lists[key]:
        if tuple[0] == document:
          score += tuple[1]
          break
        
    if score != 0:
      heapq.heappush(r, (score, document))
      
  return heapq.nlargest(k, r)   
  

In [0]:
inverted_index = build_index(data_text)

queries = ["bolsonaro", "corrupção", "economia", "brasil", "governo"]

In [0]:
def mim(intersection, a, b):
  if len(a) > 0 and len(b) > 0:
    return (len(intersection) / (len(a) * len(b)))
  
  return 0

In [0]:
def emim(intersection, a, b, N):
  if len(intersection) > 0:
    return len(intersection) * math.log10(N * mim(intersection, a, b))
  
  return 0

In [0]:
def chi_2(intersection, a, b, n):
  if len(a) > 0 and len(b) > 0:
    return ((len(intersection) - ((1/n) * len(a) * len(b))) ** 2) / (len(a) * len(b))
  
  return 0

In [0]:
def dice(intersection, a, b):
  if len(a) + len(b) == 0 :
    return 0
  return len(intersection) / (len(a) + len(b))

In [0]:
def get_intersection(a, b):
  docs_a = []
  for item in a:
    docs_a.append(item[0])
    
  
  docs_b = []
  for item in b:
    docs_b.append(item[0])
      
  return [value for value in docs_a if value in docs_b]

In [0]:
def generate_table(query, inverted_index):
  col_mim = {}
  col_emim = {}
  col_chi_squared = {}
  col_dice = {}

  a = inverted_index[query]

  for key in inverted_index.keys():
    if key != query: 
    
      b = inverted_index[key]
  
      intersection = get_intersection(a, b)
  
      mim_value = mim(intersection, a, b)
      emim_value = emim(intersection, a, b, len(inverted_index))
      chi_squared_value = chi_2(intersection, a, b, len(inverted_index))
      dice_value = dice(intersection, a, b)
    
      col_mim[mim_value] = key
      col_emim[emim_value] = key
      col_chi_squared[chi_squared_value] = key
      col_dice[dice_value] = key
    
  top10_mim = []
  top10_emim = []
  top10_chi_2 = []
  top10_dice = []
    
  for i in range(10) :
    top10_mim.append(col_mim.pop(max(col_mim)))
    top10_emim.append(col_emim.pop(max(col_emim)))
    top10_chi_2.append(col_chi_squared.pop(max(col_chi_squared)))
    top10_dice.append(col_dice.pop(max(col_dice)))

  table = {"MIM": top10_mim, "EMIM": top10_emim, "CHI-SQUARED": top10_chi_2, "DICE": top10_dice}
  df = pd.DataFrame.from_dict(table) 
  
  return df 

In [0]:
#Questão 1

In [14]:
df = generate_table('bolsonaro', inverted_index)
df

,MIM,EMIM,CHI-SQUARED,DICE
0,participativos,presidente,presidente,presidente
1,planalto,brasil,brasil,brasil
2,psl,ainda,governo,governo
3,guedes,governo,brasileiro,brasileiro
4,lula,sobre,ainda,político
5,consegue,ser,ministro,ministro
6,articulação,país,político,brasileira
7,capitão,anos,reforma,ainda
8,relevantes,ter,jair,federal
9,palácio,política,brasileira,reforma


In [15]:
df = generate_table('corrupção', inverted_index)
df

,MIM,EMIM,CHI-SQUARED,DICE
0,somavam,ainda,ex-chefe,mdb
1,email,contra,mdb,federal
2,suspeitos,federal,federal,popularidade
3,lavagem,ser,lavagem,temer
4,patrão,outros,popularidade,lavagem
5,moreira,menos,email,poderoso
6,lava,anos,bretas,confiança
7,poderoso,política,temer,garantir
8,popularidade,político,político,prisão
9,permaneceram,maior,poderoso,ex-chefe


In [16]:
df = generate_table('economia', inverted_index)
df

,MIM,EMIM,CHI-SQUARED,DICE
0,participativos,sobre,previdência,reforma
1,guedes,menos,reforma,previdência
2,maia,ser,mercado,mercado
3,aposentadoria,governo,guedes,governo
4,articulação,ainda,governo,ministro
5,emenda,presidente,menos,econômica
6,mercados,anos,econômica,menos
7,sustentável,política,política,política
8,presidir,outros,sobre,presidente
9,previdência,pode,ministro,bolsonaro


In [17]:
df = generate_table('brasil', inverted_index)
df

,MIM,EMIM,CHI-SQUARED,DICE
0,participativos,sobre,bolsonaro,bolsonaro
1,brasileiros,ainda,ainda,ainda
2,pra,ser,sobre,sobre
3,diferenças,anos,ser,governo
4,planejamento,país,governo,ser
5,acionistas,ter,brasileiros,presidente
6,simbólico,governo,anos,anos
7,denúncia,presidente,sociais,país
8,grosso,bolsonaro,presidente,sociais
9,horizonte,outros,brasileira,outros


In [18]:
df = generate_table('governo', inverted_index)
df

,MIM,EMIM,CHI-SQUARED,DICE
0,detida,sobre,presidente,presidente
1,guedes,país,país,país
2,legislativo,ser,bolsonaro,bolsonaro
3,desemprego,ainda,sobre,sobre
4,cometidos,presidente,ainda,ainda
5,sinais,anos,ser,parte
6,reformado,parte,parte,brasil
7,reformas,menos,brasil,ser
8,executivo,brasil,ministro,menos
9,incapaz,ter,menos,estado


Analisando as cinco tabelas acima, é possível afirmar que as métricas Qui-quadrado e Dice apresentaram resultados melhores em relação às outras, sendo o qui-quadrado um pouco melhor.

In [0]:
#Questão 2

A query escolhida foi 'governo'. A seguir, será realizada a consulta expandida para, respectivamente, os top 3, 5 e 10 documentos de acordo com a métrica qui-quadrado

In [0]:
top3 = document_at_a_time('governo presidente país bolsonaro', inverted_index, 10)
top5 = document_at_a_time('governo presidente país bolsonaro sobre ainda', inverted_index, 10)
top10 = document_at_a_time('governo presidente país bolsonaro sobre ainda ser parte brasil ministro menos', inverted_index, 10)


In [0]:
from IPython.display import display, HTML
pd.options.display.max_colwidth = 500

In [22]:
df = {'title': [], 'subtitle': []}
for item in top3:
  df['title'].append(data_title[item[1]])
  df['subtitle'].append(data_subtitle[item[1]])
  
df = pd.DataFrame.from_dict(df)
display(df)

,title,subtitle
0,Mas quem é essa nova direita que ganha espaço pelo mundo?,Movimento que cresce na Europa EUA e América Latina não se apresenta da mesma forma em todos os lugares mas mescla em doses distintas algumas características
1,Socialismo ‘millennial’ nos EUA,A temida palavra ‘socialista’ já não é mais tabu entre os jovens do país. Críticos da má distribuição de oportunidades e do ‘status quo’ encontraram sua voz em Alexandria Ocasio-Cortez
2,Igualdade radical,Movimento feminista impõe uma nova energia democratizante
3,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,"Com medida simbólica presidente determinou que Forças Armadas façam ""comemorações devidas"" da data"
4,Paulo Guedes cancela ida à Câmara para defender reforma da Previdência e acirra tensão com o Legislativo,Ministro da Economia ia em 'missão de paz' para explicar projeto de reforma na CCJ e tentar atenuar crise entre Governo e Rodrigo Maia. Desistência de Guedes foi criticada
5,Os sete tiros de fuzil do presidente,A guerra a das armas e das ideologias é a grama que melhor parece crescer nesse jardim sombrio do capitão reformado caçador de conflitos
6,“Mostre o que as loucas podem fazer”: o anúncio da Nike sobre as mulheres no esporte,Tenista norte-americana Serena Williams narra como as esportistas são vistas em momentos de tensão
7,Michel Temer deixa a prisão após decisão da Justiça Federal no Rio,Decisão é do desembargador Antonio Ivan Athié do TRF-2 que também mandou soltar o ex-ministro Moreira Franco e outros seis presos na Operação Descontaminação
8,Estatal espanhola fica com aeroportos do Nordeste em leilão positivo para Bolsonaro,É a maior operação internacional da estatal aeroportuária espanhola que tem 51% das ações controladas pelo Estado espanhol
9,Brasil leva susto mas vence a República Tcheca de virada,Em amistoso seleção de Tite saiu perdendo mas Firmino e Gabriel Jesus (duas vezes) marcaram os gols da vitória por 3 a 1


In [23]:
df = {'title': [], 'subtitle': []}
for item in top5:
  df['title'].append(data_title[item[1]])
  df['subtitle'].append(data_subtitle[item[1]])
  
df = pd.DataFrame.from_dict(df)
display(df)

,title,subtitle
0,Socialismo ‘millennial’ nos EUA,A temida palavra ‘socialista’ já não é mais tabu entre os jovens do país. Críticos da má distribuição de oportunidades e do ‘status quo’ encontraram sua voz em Alexandria Ocasio-Cortez
1,Mas quem é essa nova direita que ganha espaço pelo mundo?,Movimento que cresce na Europa EUA e América Latina não se apresenta da mesma forma em todos os lugares mas mescla em doses distintas algumas características
2,Igualdade radical,Movimento feminista impõe uma nova energia democratizante
3,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,"Com medida simbólica presidente determinou que Forças Armadas façam ""comemorações devidas"" da data"
4,Os sete tiros de fuzil do presidente,A guerra a das armas e das ideologias é a grama que melhor parece crescer nesse jardim sombrio do capitão reformado caçador de conflitos
5,Paulo Guedes cancela ida à Câmara para defender reforma da Previdência e acirra tensão com o Legislativo,Ministro da Economia ia em 'missão de paz' para explicar projeto de reforma na CCJ e tentar atenuar crise entre Governo e Rodrigo Maia. Desistência de Guedes foi criticada
6,Michel Temer deixa a prisão após decisão da Justiça Federal no Rio,Decisão é do desembargador Antonio Ivan Athié do TRF-2 que também mandou soltar o ex-ministro Moreira Franco e outros seis presos na Operação Descontaminação
7,Moro é mais popular que Bolsonaro em Governo que segue perdendo aprovação,Atlas Político confirma tendência de queda na popularidade da gestão e do presidente e mostra índice bom/excelente numericamente atrás de regular e ruim/péssimo. 87% são a favor da prisão de Temer
8,Estatal espanhola fica com aeroportos do Nordeste em leilão positivo para Bolsonaro,É a maior operação internacional da estatal aeroportuária espanhola que tem 51% das ações controladas pelo Estado espanhol
9,De surpresa Trump revoga as últimas sanções impostas à Coreia do Norte,A Casa Branca contradiz o Departamento do Tesouro e justifica a decisão dizendo que “o presidente Trump simpatiza com o líder” norte-coreano


In [24]:
df = {'title': [], 'subtitle': []}
for item in top10:
  df['title'].append(data_title[item[1]])
  df['subtitle'].append(data_subtitle[item[1]])
  
df = pd.DataFrame.from_dict(df)
display(df)

,title,subtitle
0,Socialismo ‘millennial’ nos EUA,A temida palavra ‘socialista’ já não é mais tabu entre os jovens do país. Críticos da má distribuição de oportunidades e do ‘status quo’ encontraram sua voz em Alexandria Ocasio-Cortez
1,Mas quem é essa nova direita que ganha espaço pelo mundo?,Movimento que cresce na Europa EUA e América Latina não se apresenta da mesma forma em todos os lugares mas mescla em doses distintas algumas características
2,Igualdade radical,Movimento feminista impõe uma nova energia democratizante
3,Os sete tiros de fuzil do presidente,A guerra a das armas e das ideologias é a grama que melhor parece crescer nesse jardim sombrio do capitão reformado caçador de conflitos
4,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,"Com medida simbólica presidente determinou que Forças Armadas façam ""comemorações devidas"" da data"
5,Paulo Guedes cancela ida à Câmara para defender reforma da Previdência e acirra tensão com o Legislativo,Ministro da Economia ia em 'missão de paz' para explicar projeto de reforma na CCJ e tentar atenuar crise entre Governo e Rodrigo Maia. Desistência de Guedes foi criticada
6,Michel Temer deixa a prisão após decisão da Justiça Federal no Rio,Decisão é do desembargador Antonio Ivan Athié do TRF-2 que também mandou soltar o ex-ministro Moreira Franco e outros seis presos na Operação Descontaminação
7,Brasil leva susto mas vence a República Tcheca de virada,Em amistoso seleção de Tite saiu perdendo mas Firmino e Gabriel Jesus (duas vezes) marcaram os gols da vitória por 3 a 1
8,De surpresa Trump revoga as últimas sanções impostas à Coreia do Norte,A Casa Branca contradiz o Departamento do Tesouro e justifica a decisão dizendo que “o presidente Trump simpatiza com o líder” norte-coreano
9,Estatal espanhola fica com aeroportos do Nordeste em leilão positivo para Bolsonaro,É a maior operação internacional da estatal aeroportuária espanhola que tem 51% das ações controladas pelo Estado espanhol


Observando os resultados acima, apesar de alguns documentos estarem presentes em todas as três consultas, é possível verificar uma nítida melhora nos resultdos referentes à consulta com os top 10 elementos,